In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy scipy
!pip install matplotlib
!pip install tensorflow
#!pip install -U scikit-learn

In [2]:
import matplotlib
import sys
sys.path.append('/content/drive/MyDrive/Age_Detection_Project/')

from model.ResNet50 import AgeEstimatorModel as ResNetModel
from model.EfficientNetB0 import AgeEstimatorModel1 as EfficientNetModel
from model.VGGNet import AgeEstimatorModel3 as VGGNetModel
from model.AlexNetV2 import AlexNet2AgeClassifier as AlexNetModel
from model.CNNmodel import AgeEstimatorModel5 as CNNModel
from model.GoogleNet_new import GoogleNetNew as GoogleNetNew
from model.VGGNet_classifier import VGG_Classifier as VGG_Classifier
import tensorflow as tf
import keras
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import glob
from scipy.stats import mode
import os.path
from torchvision import models
import torch.nn as nn
import torch
from keras.callbacks import LearningRateScheduler
from keras.optimizers.legacy import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from keras import layers
from PIL import Image
from io import BytesIO
from transformers import ViTFeatureExtractor, ViTForImageClassification
from tqdm import tqdm
from keras.utils import to_categorical

In [3]:
# Paths and parameters initialization
dataset_path = "/content/drive/MyDrive/Age_Detection_Project/Faces"
utkface_path = "/content/drive/MyDrive/Age_Detection_Project/utkcropped"
model_path_resnet = "/content/drive/MyDrive/Age_Detection_Project/resnet_age_detection.model"
model_path_effnet = "/content/drive/MyDrive/Age_Detection_Project/effnet_age_detection.model"
model_path_alexnet = "/content/drive/MyDrive/Age_Detection_Project/AlexNet2_age_detection.model"
model_path_vggnet = "/content/drive/MyDrive/Age_Detection_Project/VGGNet_0.001_age_detection.model"
model_path_googlenet = "/content/drive/MyDrive/Age_Detection_Project/GoogleNet_age_detection.model"
model_path_cnn = "/content/drive/MyDrive/Age_Detection_Project/CNNModel_age_detection.model"
plot_path_resnet = "/content/drive/MyDrive/Age_Detection_Project/resnet_plot.png"
plot_path_effnet = "/content/drive/MyDrive/Age_Detection_Project/effnet_plot.png"
plot_path_googlenet = "/content/drive/MyDrive/Age_Detection_Project/googlenet_plot.png"
plot_path_cnn = "/content/drive/MyDrive/Age_Detection_Project/cnn_plot.png"
plot_path_alexnet = "/content/drive/MyDrive/Age_Detection_Project/alexnet_plot.png"
plot_path_vggnet = "/content/drive/MyDrive/Age_Detection_Project/vggnet_plot.png"

In [ ]:
images_0_17_path ="/content/drive/MyDrive/Age_Detection_Project/0-17"
images_18_25 = "/content/drive/MyDrive/Age_Detection_Project/18-25"
images_26_50 = "/content/drive/MyDrive/Age_Detection_Project/26-50"
image_51_70 = "/content/drive/MyDrive/Age_Detection_Project/51-70"
images_71_120 = "/content/drive/MyDrive/Age_Detection_Project/71-120"

In [ ]:
# Initial parameters
epochs = 100
lr = 0.001
batch_size = 64
img_dims = (96, 96, 3)

# Load and preprocess the data
data = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(images_0_17_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths

for i in range(0,len(image_files)):
    image = cv2.imread(image_files[i])
    image = cv2.resize(image, (img_dims[0], img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    # Extract age labels from the image paths
    label = int(image_files[i].split("/")[-1].split("_")[0])
    age_labels.append(label)

print(len(data))
print(age_labels)

1935
1935
[8, 15, 9, 7, 8, 2, 16, 3, 2, 7, 3, 3, 4, 2, 3, 2, 6, 5, 3, 2, 2, 16, 6, 13, 4, 9, 1, 2, 4, 6, 5, 2, 9, 9, 5, 9, 8, 1, 9, 1, 2, 10, 1, 9, 4, 4, 16, 8, 4, 1, 7, 2, 9, 16, 5, 4, 7, 7, 3, 2, 4, 16, 14, 3, 3, 7, 2, 6, 8, 9, 8, 15, 1, 8, 5, 2, 2, 4, 6, 5, 5, 1, 7, 4, 6, 4, 3, 3, 15, 3, 4, 12, 10, 3, 2, 5, 5, 3, 3, 7, 8, 2, 1, 8, 2, 1, 9, 14, 11, 8, 7, 9, 8, 7, 4, 6, 3, 2, 8, 1, 7, 14, 4, 9, 6, 6, 2, 16, 9, 2, 12, 3, 8, 16, 2, 5, 8, 2, 1, 4, 3, 5, 1, 8, 5, 6, 7, 2, 8, 14, 6, 14, 2, 3, 4, 3, 2, 8, 11, 17, 5, 3, 13, 10, 3, 1, 9, 5, 2, 4, 3, 5, 8, 1, 3, 3, 2, 2, 3, 16, 1, 4, 4, 4, 7, 1, 9, 11, 4, 15, 1, 9, 4, 7, 11, 2, 4, 3, 1, 17, 2, 9, 3, 8, 6, 15, 3, 6, 2, 6, 6, 9, 2, 9, 4, 2, 6, 4, 2, 3, 5, 8, 9, 4, 8, 17, 4, 2, 8, 2, 9, 2, 2, 16, 2, 5, 7, 8, 7, 2, 4, 4, 2, 3, 2, 16, 4, 2, 2, 4, 5, 1, 2, 7, 9, 2, 7, 7, 8, 2, 7, 16, 8, 6, 16, 1, 3, 8, 9, 6, 1, 3, 2, 8, 2, 2, 5, 3, 2, 17, 4, 8, 8, 2, 3, 4, 2, 8, 8, 4, 7, 2, 4, 11, 5, 9, 1, 2, 3, 2, 5, 2, 4, 2, 1, 5, 2, 1, 7, 3, 1, 4, 16, 5, 1, 3, 4,

In [ ]:
# Pre-process the data
data = np.array(data, dtype="float") / 255.0
labels = np.array(age_labels)


print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


Data shape: (1935, 96, 96, 3)
Labels shape: (1935,)


In [ ]:
# build and train ResNet50 model
model_resnet = ResNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
#H_resnet = train_model(model_resnet, "ResNet50")

# Compile the model
optimizer = Adam(lr=0.001)
model_resnet.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_resnet.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)

pred = model_resnet.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for resnet", mae)

94765736/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 4.7953 - mae: 4.7953
Epoch 1: val_loss improved from inf to 22602.95312, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 282s 11s/step - loss: 4.7953 - mae: 4.7953 - val_loss: 22602.9531 - val_mae: 22602.9531
Epoch 2/100
25/25 [==============================] - ETA: 0s - loss: 2.8206 - mae: 2.8206
Epoch 2: val_loss improved from 22602.95312 to 1721.51453, saving model to best_model.h5
25/25 [==============================] - 262s 11s/step - loss: 2.8206 - mae: 2.8206 - val_loss: 1721.5145 - val_mae: 1721.5145
Epoch 3/100
25/25 [==============================] - ETA: 0s - loss: 2.5474 - mae: 2.5474 
Epoch 3: val_loss improved from 1721.51453 to 7.36769, saving model to best_model.h5
25/25 [==============================] - 267s 11s/step - loss: 2.5474 - mae: 2.5474 - val_loss: 7.3677 - val_mae: 7.3677
Epoch 4/100
25/25 [==============================] - ETA: 0s - loss: 2.5294 - mae: 2.5294
Epoch 4: val_loss improved from 7.36769 to 3.25909, saving model to best_model.h5
25/25 [==============================] - 254s 10s/step - loss: 2.5294 - mae: 2.5294 - val_loss: 

In [ ]:
# build and train EfficientNetB0 model
model_effnet = EfficientNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
#H_effnet = train_model(model_effnet, "EfficientNetB0")

# Compile the model
optimizer = Adam(lr=0.001)
model_effnet.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_effnet.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)

pred = model_effnet.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for effnet", mae)

In [ ]:
# build and train GoogleNet model
model_googlenet = GoogleNetNew.GoogLeNet()
#H_googlenet = train_model(model_googlenet, "GoogleNet")
# Compile the model
optimizer = Adam(lr=0.0001)
model_googlenet.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_googlenet.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)

pred = model_googlenet.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for googlenet", mae)

In [ ]:
# build and train VGGNet model
model_vggnet = VGGNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
#H_vggnet = train_model(model_vggnet, "VGGNet")

# Compile the model
optimizer = Adam(lr=0.001)
model_vggnet.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_vggnet.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)


pred = model_vggnet.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for vggnet", mae)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
33/33 [==============================] - ETA: 0s - loss: 45.3679 - mae: 45.3679
Epoch 1: val_loss improved from inf to 6.81034, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


33/33 [==============================] - 11s 320ms/step - loss: 45.3679 - mae: 45.3679 - val_loss: 6.8103 - val_mae: 6.8103
Epoch 2/100
33/33 [==============================] - ETA: 0s - loss: 5.6932 - mae: 5.6932
Epoch 2: val_loss improved from 6.81034 to 5.07219, saving model to best_model.h5
33/33 [==============================] - 6s 171ms/step - loss: 5.6932 - mae: 5.6932 - val_loss: 5.0722 - val_mae: 5.0722
Epoch 3/100
33/33 [==============================] - ETA: 0s - loss: 5.5336 - mae: 5.5336
Epoch 3: val_loss improved from 5.07219 to 4.90194, saving model to best_model.h5
33/33 [==============================] - 6s 168ms/step - loss: 5.5336 - mae: 5.5336 - val_loss: 4.9019 - val_mae: 4.9019
Epoch 4/100
33/33 [==============================] - ETA: 0s - loss: 5.8753 - mae: 5.8753
Epoch 4: val_loss did not improve from 4.90194
33/33 [==============================] - 5s 159ms/step - loss: 5.8753 - mae: 5.8753 - val_loss: 5.1939 - val_mae: 5.1939
Epoch 5/100
33/33 [=============

In [ ]:
# build and train AlexNet model
model_alexnet2 = AlexNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
#H_alexnet2 = train_model(model_alexnet2, "AlexNet2")
# Compile the model
optimizer = Adam(lr=0.001)
model_alexnet2.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_alexnet2.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)


pred = model_alexnet2.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for AlexNet", mae)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
33/33 [==============================] - ETA: 0s - loss: 57.7825 - mae: 57.7825
Epoch 1: val_loss improved from inf to 56.81759, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


33/33 [==============================] - 8s 214ms/step - loss: 57.7825 - mae: 57.7825 - val_loss: 56.8176 - val_mae: 56.8176
Epoch 2/100
33/33 [==============================] - ETA: 0s - loss: 53.3518 - mae: 53.3518
Epoch 2: val_loss improved from 56.81759 to 50.42168, saving model to best_model.h5
33/33 [==============================] - 6s 178ms/step - loss: 53.3518 - mae: 53.3518 - val_loss: 50.4217 - val_mae: 50.4217
Epoch 3/100
33/33 [==============================] - ETA: 0s - loss: 44.1876 - mae: 44.1876
Epoch 3: val_loss improved from 50.42168 to 38.72923, saving model to best_model.h5
33/33 [==============================] - 6s 182ms/step - loss: 44.1876 - mae: 44.1876 - val_loss: 38.7292 - val_mae: 38.7292
Epoch 4/100
33/33 [==============================] - ETA: 0s - loss: 30.7674 - mae: 30.7674
Epoch 4: val_loss did not improve from 38.72923
33/33 [==============================] - 5s 148ms/step - loss: 30.7674 - mae: 30.7674 - val_loss: 907.3552 - val_mae: 907.3552
Epoch 

In [ ]:
# Compile the model
model_cnn = CNNModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2])
optimizer = Adam(lr=0.001)
model_cnn.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
# Train the model with callbacks
model_cnn.fit(trainX, trainY,
                    batch_size=64,
                    epochs=100,
                    validation_data=(testX, testY),
                    callbacks=callbacks)

pred = model_cnn.predict(np.array(testX))
mae = mean_absolute_error(testY, pred)

print("Mean Absolute Error for CNN", mae)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
33/33 [==============================] - ETA: 0s - loss: 16.0386 - mae: 16.0386
Epoch 1: val_loss improved from inf to 8.74715, saving model to best_model.h5
33/33 [==============================] - 2s 49ms/step - loss: 16.0386 - mae: 16.0386 - val_loss: 8.7471 - val_mae: 8.7471
Epoch 2/100
13/33 [==========>...................] - ETA: 0s - loss: 7.8743 - mae: 7.8743

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/33 [===========================>..] - ETA: 0s - loss: 7.8590 - mae: 7.8590
Epoch 2: val_loss improved from 8.74715 to 7.05171, saving model to best_model.h5
33/33 [==============================] - 0s 13ms/step - loss: 7.8306 - mae: 7.8306 - val_loss: 7.0517 - val_mae: 7.0517
Epoch 3/100
31/33 [===========================>..] - ETA: 0s - loss: 6.9744 - mae: 6.9744
Epoch 3: val_loss did not improve from 7.05171
33/33 [==============================] - 0s 12ms/step - loss: 6.9389 - mae: 6.9389 - val_loss: 8.1696 - val_mae: 8.1696
Epoch 4/100
31/33 [===========================>..] - ETA: 0s - loss: 6.5289 - mae: 6.5289
Epoch 4: val_loss improved from 7.05171 to 5.72368, saving model to best_model.h5
33/33 [==============================] - 0s 13ms/step - loss: 6.5336 - mae: 6.5336 - val_loss: 5.7237 - val_mae: 5.7237
Epoch 5/100
31/33 [===========================>..] - ETA: 0s - loss: 6.1913 - mae: 6.1913
Epoch 5: val_loss improved from 5.72368 to 5.16669, saving model to best_model.h5

In [ ]:
XXXXXX.save("MODELNAME_CLASS.keras")

In [ ]:
def image_preprocessing(img_path):

 #Read the image
 img=cv2.imread(img_path)

 #Resize the image
 resized_img=cv2.resize(img,(96,96))

 #Normalize the image
 normalized_img=resized_img/255
 #normalized_img_batch = np.expand_dims(normalized_img, axis=0)

 return normalized_img

In [ ]:
def label_extraction(img_name):
 #Extract age
  age=int(img_name.split("/")[-1].split("_")[0])
  if 0<age<18 :
    class_index=0
  elif 17<age<26:
    class_index=1
  elif 25<age<51:
    class_index=2
  elif 50<age<71:
    class_index=3
  else:
    class_index=4
  return class_index

In [ ]:
image_files = [f for f in glob.glob(utkface_path + "/*.jpg", recursive=True)]

In [ ]:
features=[]
labels=[]

for img_name in image_files:
  class_index=label_extraction(img_name)
  labels.append(class_index)

  preprocessed_img=image_preprocessing(img_name)
  #Append data into appropriate lists
  features.append(preprocessed_img)



#Convert lists to numpy arrays
features=np.asarray(features)
labels=np.asarray(labels)

In [ ]:
encoded_labels=to_categorical(labels)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, encoded_labels,test_size = 0.1, shuffle = True,random_state = 42)

In [ ]:
vgg_classifier = VGG_Classifier.build(96,96,3,5)

optimizer = Adam(lr=0.001)
vgg_classifier.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model with learning rate scheduler callback
history = vgg_classifier.fit(x = features_train, y = labels_train, epochs = 50, shuffle = True, validation_split = 0.1, callbacks = callbacks )


model_evaluation_history = vgg_classifier.evaluate(features_test, labels_test)

Epoch 1/50
343/343 [==============================] - ETA: 0s - loss: 1.7723 - accuracy: 0.5609
Epoch 1: val_loss improved from inf to 1.31240, saving model to best_model.h5
343/343 [==============================] - 12s 18ms/step - loss: 1.7723 - accuracy: 0.5609 - val_loss: 1.3124 - val_accuracy: 0.5865
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


341/343 [============================>.] - ETA: 0s - loss: 1.2353 - accuracy: 0.6163
Epoch 2: val_loss improved from 1.31240 to 1.05338, saving model to best_model.h5
343/343 [==============================] - 5s 13ms/step - loss: 1.2353 - accuracy: 0.6165 - val_loss: 1.0534 - val_accuracy: 0.6505
Epoch 3/50
341/343 [============================>.] - ETA: 0s - loss: 1.1173 - accuracy: 0.6304
Epoch 3: val_loss improved from 1.05338 to 0.99088, saving model to best_model.h5
343/343 [==============================] - 5s 13ms/step - loss: 1.1177 - accuracy: 0.6300 - val_loss: 0.9909 - val_accuracy: 0.6710
Epoch 4/50
341/343 [============================>.] - ETA: 0s - loss: 1.0702 - accuracy: 0.6298
Epoch 4: val_loss improved from 0.99088 to 0.96874, saving model to best_model.h5
343/343 [==============================] - 5s 13ms/step - loss: 1.0706 - accuracy: 0.6296 - val_loss: 0.9687 - val_accuracy: 0.6620
Epoch 5/50
341/343 [============================>.] - ETA: 0s - loss: 1.0323 - ac

In [ ]:
vgg_classifier.save("VGG_classifier.keras")

In [4]:
model_resnet_0 = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/ResNet_0_17.keras")
model_resnet_1 = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/ResNet_26_50.keras")
model_effnet = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/EfficientNet_18_25.keras")
model_cnn = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/CNN_51_70.keras")
model_vggnet = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/VGG_71_120.keras")
vgg_classifier = tf.keras.models.load_model("/content/drive/MyDrive/Age_Detection_Project/VGG_classifier.keras")

In [5]:
test_images = [f for f in glob.glob("/content/drive/MyDrive/Age_Detection_Project/test_images" + "/*.jpg", recursive=True)]

In [7]:
preds_mae = []
for image in test_images:
  img = cv2.imread(image)
  age=int(image.split("/")[-1].split("_")[2])
  # Redimensionner l'image
  resized_img = cv2.resize(img, (96, 96))
  # Normaliser l'image
  normalized_img = resized_img / 255.0

  # Ajouter une dimension de lot supplémentaire
  normalized_img_batch = np.expand_dims(normalized_img, axis=0)

  predictions = np.array(vgg_classifier.predict(normalized_img_batch))
  max_index = np.argmax(predictions)

  if max_index == 0:
    pred = model_resnet_0.predict(normalized_img_batch)
    preds_mae.append(np.abs(age - pred))
  elif max_index == 1:
    pred = model_effnet.predict(normalized_img_batch)
    preds_mae.append(np.abs(age - pred))
  elif max_index == 2:
    pred = model_resnet_1.predict(normalized_img_batch)
    preds_mae.append(np.abs(age - pred))
  elif max_index == 3:
    pred = model_cnn.predict(normalized_img_batch)
    preds_mae.append(np.abs(age - pred))
  elif max_index == 4:
    pred = model_vggnet.predict(normalized_img_batch)
    preds_mae.append(np.abs(age - pred))

mae_cascade = np.mean(preds_mae)

print("MAE for cascade architecture: ", mae_cascade)

1/1 [==============================] - 0s 22ms/step
MAE for cascade architecture:  14.332493
